# Setup

In [42]:
# TODO: add annotations describing usage of different modules

from operator import mod
from os import getcwd
from os.path import exists, join

import joblib
from sklearn.datasets import fetch_california_housing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVR
import pandas as pd
import numpy as np
# from ydata_profiling import ProfileReport

from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.linear_model import LogisticRegression, LinearRegression
import warnings
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import  GradientBoostingClassifier
# import xgboost as xgb
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC, LinearSVC 
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold

from sklearn.metrics import recall_score

from sklearn import tree
from sklearn.decomposition import PCA, SparsePCA

from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
import json
import pickle
from IPython.display import Image
import warnings

from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from collections import Counter

import altair as alt
import random
import warnings

warnings.filterwarnings('ignore')

# set seaborn whitegrid theme
sns.set(style="whitegrid")

from sklearn.inspection import permutation_importance
from random import sample

from sklearn.impute import KNNImputer
from sklearn.model_selection import RepeatedStratifiedKFold


from tqdm import tqdm

tqdm.pandas()

In [43]:
def get_classification_report(pred_labels, y_test, algo_name, show_full_report=False):
    eval_on = 'macro avg' 
    # eval_on = 'depressed' #not_depressed
    target_names = ['not_depressed', 'depressed',]
    
    df_cls_rpt = pd.DataFrame(
        classification_report(
            y_test, 
            pred_labels, 
            target_names=target_names, 
            output_dict=True
        )
    ).rename_axis('metric')\
    .reset_index()

    
    if show_full_report == True:
        display(df_cls_rpt)
    accuracy = df_cls_rpt[['accuracy']].iloc[0, 0]
    df_cls_rpt = df_cls_rpt[['metric', eval_on]].T
    df_cls_rpt.columns = df_cls_rpt.iloc[0,:]
    df_cls_rpt = df_cls_rpt.iloc[1:,:]
    df_cls_rpt['accuracy'] = accuracy

    df_cls_rpt['algo'] = algo_name

    first_column = df_cls_rpt.pop('algo')
    df_cls_rpt.insert(0, 'algo', first_column)

    # display(df_cls_rpt)
    return df_cls_rpt

In [44]:
def algo_grid_search_pred(
    X_train, 
    y_train, 
    X_test,
    y_test,
    algo,
    algo_name,
    params,
    cv,
    verbose,
    imputer,
    scaler,
    score_on):

    if score_on == 'f1-score':
        score_on = 'f1_macro'
    if algo_name == 'Multinomial Naive Bayes':
        X_train[X_train < 0] = 0
        X_test[X_test < 0] = 0
        X_train = np.nan_to_num(X_train)
        X_test = np.nan_to_num(X_test)

    pipeline_list = [imputer, scaler, algo]
    # drop imputer or scaler if none
    pipeline_list = [step for step in pipeline_list if step is not None]

    processing_pipeline = make_pipeline(imputer, scaler, algo)
    processing_pipeline = make_pipeline()
    i = 1
    for step in pipeline_list:
        processing_pipeline.steps.append([(type(step).__name__).lower(), step])
        i += 1
    grid = GridSearchCV(
        processing_pipeline, 
        param_grid=params, 
        n_jobs=-1, 
        cv=cv, 
        verbose=verbose, 
        scoring=score_on)
    if algo_name == 'gnb':
        X_test = PowerTransformer().fit_transform(X_test)

        # grid.fit(X_test, y_test)
    grid.fit(X_train, y_train)

    pred_labels = [x.round() for x in grid.best_estimator_.predict(X_test)]
    pred_labels = [x.round() for x in pred_labels]

    df_algo_cls_rpt = get_classification_report(pred_labels, y_test, algo_name)
    df_algo_cls_rpt['train_r2'] = grid.best_estimator_.score(X_train, y_train)
    df_algo_cls_rpt['test_r2'] = grid.best_estimator_.score(X_test, y_test)
    df_algo_cls_rpt['best_params'] = str(grid.best_params_)
    tn, fp, fn, tp = confusion_matrix(pred_labels, y_test).ravel()
    df_algo_cls_rpt['tp'] = tp
    df_algo_cls_rpt['fn'] = fn
    df_algo_cls_rpt['fp'] = fp
    df_algo_cls_rpt['tn'] = tn

    
    return df_algo_cls_rpt, pred_labels

In [45]:
def algo_baseline_pred(
    X_train, 
    y_train, 
    X_test,
    y_test,
    show_full_report,
    algo,
    algo_name,
    imputer,
    scaler):
    
    if algo_name == 'Multinomial Naive Bayes':
        X_train[X_train < 0] = 0
        X_test[X_test < 0] = 0
        X_train = np.nan_to_num(X_train)
        X_test = np.nan_to_num(X_test)

    pipeline_list = [imputer, scaler, algo]
    # drop imputer or scaler if none
    pipeline_list = [step for step in pipeline_list if step is not None]
    
    try:
        processing_pipeline = make_pipeline()
        i = 1
        for step in pipeline_list:
            processing_pipeline.steps.append([(type(step).__name__).lower(), step])
            i += 1

        processing_pipeline.fit(X_train, y_train)
        pred_labels  = processing_pipeline.predict(X_test)
        pred_labels = [x.round() for x in pred_labels]

        df_algo_cls_rpt = get_classification_report(pred_labels, y_test, algo_name, show_full_report)
        tn, fp, fn, tp = confusion_matrix(y_test, pred_labels).ravel()
        df_algo_cls_rpt['tp'] = tp
        df_algo_cls_rpt['fn'] = fn
        df_algo_cls_rpt['fp'] = fp
        df_algo_cls_rpt['tn'] = tn
        
        return df_algo_cls_rpt, pred_labels
    except ValueError as e:
        print(e)
        raise Exception (f'{algo_name} might not work with NaN')
        
    return
    

In [46]:


def baseline_models(
    X_train, 
    y_train, 
    X_test, 
    y_test,
    algo_attempt_list,
    do_smote=False,
    show_confusion_matrix=False,
    show_full_report=False,
    grid_search=False,
    cv=5,
    verbose=0,
    imputer=SimpleImputer(),
    scaler=RobustScaler(),
    bin_vars=False,
    score_on='recall'):
    """

    """


    # do_smote
    if do_smote == True:
        # have to impute first because smote won't take nulls
        my_imputer = SimpleImputer()
        X_train = my_imputer.fit_transform(X_train)
        X_test = my_imputer.fit_transform(X_test)

        sm = SMOTE(random_state=42)
        X_train, y_train = sm.fit_resample(X_train, y_train)

    # if bin_vars == True:
    #     est = KBinsDiscretizer(
    #         n_bins=3,
    #         encode='ordinal', 
    #         strategy='uniform', 
    #         subsample=None
    #     )
    #     my_imputer = SimpleImputer()
    #     X_train = my_imputer.fit_transform(X_train)
    #     X_test = my_imputer.fit_transform(X_test)
        
    #     est.fit(X_train)
    #     X_train = est.transform(X_train)

    #     est.fit(X_test)
    #     X_test = est.transform(X_test)

    df_cls_rpt = pd.DataFrame()
    conf_mtrx_dict = {}

    if algo_attempt_list == 'all':
        algo_attempt_list = ['knn', 'lm', 'bnb', 'gnb', 'dt', 'rf', 'gb', 'xgb']
    
    for a in algo_attempt_list:
        # SVR
        if a == 'svr':
            algo_name = 'SVR'
            params = {
                "simpleimputer__strategy": ["mean", "median"],
                "robustscaler__quantile_range": [(25.0, 75.0), (30.0, 70.0)],
                "svr__C": [0.1, 1.0],
                "svr__gamma": ["auto", 0.1],
            }
            algo = SVR()
        elif a == 'knn':
            algo_name = 'KNN'
            params = {
                'kneighborsclassifier__n_neighbors': list(range(1, 15))
            }
            print(params)
            algo = KNeighborsClassifier()
        elif a == 'lm':
            algo_name = 'Logistic Regression'
            params = {
                'logisticregression__penalty': ['l1','l2'], 
                'logisticregression__C': [0.001,0.01,0.1,1,10,100,1000]
            }
            algo = LogisticRegression(max_iter=1000, penalty='l2', C=10)
        elif a == 'bnb':
            algo_name = 'Bernoulli Naive Bayes'
            params = {
                'bernoullinb__alpha': np.logspace(0,-9, num=100),
                'bernoullinb__binarize': [0.0, 1.0, 2.0],
                'bernoullinb__fit_prior': [True, False]
            }
            algo = BernoulliNB()
        elif a == 'gnb':
            algo_name = 'Gaussian Naive Bayes'
            params = {
                'gaussiannb__var_smoothing': np.logspace(0,-9, num=100)
            }
            algo = GaussianNB()
        elif a == 'mnb':
            algo_name = 'Multinomial Naive Bayes'
            params = {
                'multinomialnb__alpha': np.logspace(0,-9, num=100),
            }
            algo = MultinomialNB()
            
        elif a == 'dt':
            algo_name = 'Decision Tree'
            params = {
                'decisiontreeclassifier__criterion':['gini', 'entropy', 'logloss'],
                'decisiontreeclassifier__max_depth': np.arange(1, 15)
            }
            algo = DecisionTreeClassifier(random_state=42)

        elif a == 'rf':
            algo_name = 'Random Forest'
            params = {
                'randomforestclassifier__criterion': ['gini', 'entropy', 'logloss'],
                "randomforestclassifier__n_estimators": [10, 20, 40, 80, 100, 125, 150],
                "randomforestclassifier__max_features": ["sqrt", "log2", None],
                "randomforestclassifier__min_samples_split": [1, 2, 4, 8],
                "randomforestclassifier__bootstrap": [True, False],
            }
            algo = RandomForestClassifier(random_state=42)
        elif a == 'gb':
            algo_name = 'Gradient Boosting Classifier'
            params = {
                "gradientboostingclassifier__loss":["log_loss", "exponential"],
                "gradientboostingclassifier__learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
                # "gradientboostingclassifier__min_samples_split": np.linspace(0.1, 0.5, 12),
                # "gradientboostingclassifier__min_samples_leaf": np.linspace(0.1, 0.5, 12),
                "gradientboostingclassifier__max_depth":[3,5,8],
                "gradientboostingclassifier__max_features":["log2", "sqrt"],
                "gradientboostingclassifier__criterion": ["friedman_mse", "mae"],
                # "gradientboostingclassifier__subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
                "gradientboostingclassifier__n_estimators":[10, 25, 50, 100, 125, 150]
            }
            algo = GradientBoostingClassifier()
        elif a == 'xgb':
            algo_name = 'XGBoost'
            params = {
                'xgbclassifier__max_depth': [3, 4, 5, 6, 8, 10, 12, 15],
                'xgbclassifier__learning_rate': [0.001, 0.01, 0.1, 0.20, 0.25, 0.30],
                "xgbclassifier__gamma":[0, 0.25, 0.5, 0.75,1],
                'xgbclassifier__n_estimators': [100, 500, 1000],
            }
            algo = xgb.XGBClassifier()
        elif a == 'mlp':
            # X = [[0., 0.], [1., 1.]]
            # y = [0, 1]
            algo_name = 'MLP'
            algo = MLPClassifier(
                max_iter=1000
                # solver='lbfgs', 
                # alpha=1e-5,
                # hidden_layer_sizes=(5, 2),
                # random_state=42,
            )
            params = {
                'mlpclassifier__solver' : ['lbfgs', 'sgd', 'adam'],
                'mlpclassifier__activation' : ['identity', 'logistic', 'tanh', 'relu'],
                'mlpclassifier__early_stopping' : [True, False]
            }

        else:
            raise Exception(f'{a} is not a supported algorithm')
            # print(f'{a} is not a supported algorithm')

        if grid_search == True and params is not None:
            df_algo_cls_rpt, pred_labels = algo_grid_search_pred(
                X_train, 
                y_train, 
                X_test,
                y_test,
                algo=algo,
                algo_name=algo_name,
                params=params,
                imputer=imputer,
                scaler=scaler,
                cv=cv,
                verbose=verbose,
                score_on=score_on
            )
            
        else:
            if grid_search == True and params is None:
                # print(f'Params not defined for {algo_name} GridSearch. Fitting baseline model.')
                pass
            df_algo_cls_rpt, pred_labels = algo_baseline_pred(
                X_train, 
                y_train, 
                X_test,
                y_test,
                show_full_report,
                algo=algo,
                algo_name=algo_name,
                imputer=imputer,
                scaler=scaler,
            )

        df_cls_rpt = pd.concat([
                df_algo_cls_rpt, 
                df_cls_rpt], 
                ignore_index=True
        )
        conf_mtrx_dict[algo_name] = pred_labels
    

    if show_confusion_matrix:
        for k,v in conf_mtrx_dict.items():
            print(f'{k} Confusion Matrix')
            plot_confusion_matrix(y_test, v)

    df_cls_rpt.sort_values(by=['f1-score'], ascending=False, inplace=True)
    df_cls_rpt = df_cls_rpt.reset_index(drop=True)
    return df_cls_rpt



In [47]:
cdc_survey = pd.read_csv('../../data/cdc_nhanes_survey_responses_clean.csv')
print(cdc_survey.shape)

# filter to pregnant moms
cdc_survey_pmom = cdc_survey[cdc_survey['has_been_pregnant'] == 1]
print(cdc_survey_pmom.shape)

(35706, 863)
(7741, 863)


In [48]:
# Depression screener
dep_screener_cols = [
    'little_interest_in_doing_things',
    'feeling_down_depressed_hopeless',
    'trouble_falling_or_staying_asleep',
    'feeling_tired_or_having_little_energy',
    'poor_appetitie_or_overeating',
    'feeling_bad_about_yourself',
    'trouble_concentrating',
    'moving_or_speaking_to_slowly_or_fast',
    'thoughts_you_would_be_better_off_dead',
    'difficult_doing_daytoday_tasks'
]

# Alcohol & smoking
alcohol_n_smoking_cols = [
    'has_smoked_tabacco_last_5days',
    'alcoholic_drinks_past_12mo', 
    'drank_alc',
    'alc_drinking_freq',
    'alc_per_day',
    'times_with_4or5_alc',
    'times_with_8plus_alc',
    'times_with_12plus_alc',
    '4plus_alc_daily',
    'days_4plus_drinks_occasion',
    'smoked_100_cigs',
    'currently_smoke'
]

#Medical Conditions 
med_condition_cols = [
    'chest_discomfort',
    'asthma',
    'anemia_treatment',
    'blood_transfusion',
    'arthritis',
    'heart_failure',
    'coronary_heart_disease',
    'angina_pectoris',
    'heart_attack',
    'stroke',
    'thyroid_issues',
    'respiratory_issues',
    'abdominal_pain',
    'gallstones',
    'gallbladder_surgery',
    'cancer',
    'dr_recommend_lose_weight',
    'dr_recommend_exercise',
    'dr_recommend_reduce_salt',
    'dr_recommend_reduce_fat',
    'currently_losing_weight',
    'currently_increase_exercise',
    'currently_reducing_salt',
    'currently_reducing_fat',
    'metal_objects',
    'has_diabetes',    
    #Blood Pressure & Cholesterol 
    'high_bp',
    'age_hypertension',
    'hypertension_prescription',
    'high_bp_prescription',
    'high_cholesterol',
    'cholesterol_prescription',
    # general health
    'has_overweight_diagnosis',   
    'height_in',
    'weight_lbs', 
    #'weight_lbs_over_height_in_ratio',
    'general_health_condition',      
]

# diet, nutrition, & exercise 
lifestyle_cols = [
    # exercise/weight loss
    'vigorous_work',
    'walk_or_bicycle',
    'vigorous_recreation',
    'moderate_recreation',    
    'count_days_moderate_recreational_activity',   
    'count_minutes_moderate_recreational_activity',
    'count_minutes_moderate_sedentary_activity',    
    'count_lost_10plus_pounds',
    'has_tried_to_lose_weight_12mo', 
    'attempt_weight_loss_1yr',           
    # diet
    'how_healthy_is_your_diet',    
    'milk_consumption_freq',
    'govmnt_meal_delivery',
    'nonhomemade_meals',
    'fastfood_meals',
    'readytoeat_meals',
    'frozen_pizza'   
]

# Reproductive Health 
reproductive_health_cols = [
    'regular_periods',
    'age_last_period',
    'try_pregnancy_1yr',
    'see_dr_fertility',
    'pelvic_infection',
    'pregnant_now',
    'pregnancy_count',
    'diabetes_pregnancy',
    'delivery_count',
    'live_birth_count',
    'age_at_first_birth',
    'age_at_last_birth',
    'months_since_birth',
    'horomones_not_bc'
]

# hospital usage & access to care
hospital_utilization_n_access_cols = [
    'general_health',
    'regular_healthcare_place',
    'time_since_last_healthcare',
    'overnight_in_hospital',
    'seen_mental_health_professional',
    'count_days_seen_doctor_12mo',
    'duration_last_healthcare_visit',   
]

# socioeconomic status
socioeconomic_cols = [
    #Food Security
    'emergency_food_received',
    'food_stamps_used',      
    #Health Insurance
    'have_health_insurance',
    'have_private_insurance',
    'plan_cover_prescriptions',   
    #Income
    'family_poverty_level',
    'family_poverty_level_category',
    #Occupation
    'hours_worked',
    'over_35_hrs_worked',
    'work_schedule'    
]

demographic_cols = [
    'food_security_level_household',   
    'food_security_level_adult',    
    'monthly_poverty_index_category',
    'monthly_poverty_index',
    'count_hours_worked_last_week',
    'age_in_years',   
    'education_level',
    'is_usa_born',    
    'has_health_insurance',
    'has_health_insurance_gap'   
]

# combine all candidate features
candidate_features = dep_screener_cols + \
    alcohol_n_smoking_cols + \
    med_condition_cols + \
    hospital_utilization_n_access_cols + \
    lifestyle_cols + \
    socioeconomic_cols + \
    reproductive_health_cols + \
    demographic_cols


len(candidate_features)

116

In [49]:
def get_model_data(original_df, 
                   columns, 
                   test_size_to_use=0.2,
                   drop_null_rows=False,
                   null_imputer_strategy='median', # mean, median, most_frequent
                   use_value_scaler=True,
                   use_smote=False):
    """
    Function to build feature & indicator matrices for both train & test.
    """
    
    # add target column (MDD)
    cols_to_use = columns.copy()
    cols_to_use.insert(0, 'MDD')
    
    df_to_use = original_df[cols_to_use]
    
    if drop_null_rows:
        df_to_use.dropna(inplace=True)
    
    # Create test & train data
    x = df_to_use.iloc[:,1:].values
    y = df_to_use['MDD'].values
    
    if not drop_null_rows:
        # SimpleImputer() = fill in missing values
        # note imputer may drop columns if no values exist for it
        imputer = SimpleImputer(strategy=null_imputer_strategy)  
        x = imputer.fit_transform(x)

    # RobustScaler() = scale features to remove outliers
    if use_value_scaler:
        trans = RobustScaler()
        x = trans.fit_transform(x)

    x_train, x_test, y_train, y_test = train_test_split(
        x, 
        y, 
        test_size=test_size_to_use, 
        random_state=42
    ) 
    
    # Technique to de-risk from positive class imbalance
    if use_smote:
        sm = SMOTE(random_state=42)
        x_train, y_train = sm.fit_resample(x_train, y_train)
    
    return x_train, x_test, y_train, y_test

In [50]:
def get_performance_df(label_actual, label_pred, model_name):
    """
    Function to calculate performance metrics for model.
    Includes precision, recal, F1, & support.
    """
    # create classification report
    result_table = classification_report(label_actual, label_pred, output_dict=True)
    result_table = pd.DataFrame.from_dict(result_table)

    # store for later
    accuracies = result_table['accuracy'][0]
    
    column_key = {
        '0':'depressed_no',
         '1':'depressed_yes',
         'accuracy':'accuracy',
         'macro avg':'macro_avg',
         'weighted avg':'weighted_avg'
    }

    # rename grouping
    result_table.columns = [column_key.get(key) for key in result_table.columns]

    # create dataframe with 1 row per grouping
    result_table.drop(labels = 'accuracy', axis = 1, inplace=True)
    result_table = result_table.transpose()
    result_table['accuracy'] = [accuracies for i in range(result_table.shape[0])]
    result_table = result_table.reset_index()
    result_table.rename(columns = {'index':'grouping'},inplace=True)
    result_table['model'] = model_name
    result_table = result_table[['model','grouping','precision','recall','f1-score','support','accuracy']]
    return result_table


def generate_models(
    x_train, 
    y_train, 
    x_test, 
    y_test):
    """
    Function that trains and makes predictions using 5 of the classifiers went over during the class.
    Meant as a helper function for easier testing of different modeling pipelines.
    """

    # K-Nearest Neighbors
    knn = KNeighborsClassifier()
    knn.fit(x_train, y_train)
    # test
    pred_labels_knn  = knn.predict(x_test)
    score_knn = get_performance_df(y_test, pred_labels_knn,'Knn')
    # train
    pred_labels_knn_train  = knn.predict(x_train)
    score_knn_train = get_performance_df(y_train, pred_labels_knn_train,'Knn')
    
    # Logistic Regression
    lm = LogisticRegression()
    lm.fit(x_train, y_train)
    # test 
    pred_labels_lr  = lm.predict(x_test)
    score_lr = get_performance_df(y_test, pred_labels_lr,'Logistic Regression')
    # train 
    pred_labels_lr_train  = lm.predict(x_train)
    score_lr_train = get_performance_df(y_train, pred_labels_lr_train,'Logistic Regression')    
        
    # Bernoulii Naive Bayes
    bnb = BernoulliNB()
    bnb.fit(x_train, y_train)
    # test 
    pred_labels_bnb  = bnb.predict(x_test)
    score_bnb = get_performance_df(y_test, pred_labels_bnb,'Bernoulli Naive Bayes')    
    # train 
    pred_labels_bnb_train  = bnb.predict(x_train)
    score_bnb_train = get_performance_df(y_train, pred_labels_bnb_train,'Bernoulli Naive Bayes')       
        
    # Gaussian Naive Bayes
    gnb = GaussianNB()
    gnb.fit(x_train, y_train)
    # test 
    pred_labels_gnb  = gnb.predict(x_test)
    score_gnb = get_performance_df(y_test, pred_labels_gnb,'Gaussian Naive Bayes')  
    # train 
    pred_labels_gnb_train  = gnb.predict(x_train)
    score_gnb_train = get_performance_df(y_train, pred_labels_gnb_train,'Gaussian Naive Bayes')         

    # Random Forest
    rf = RandomForestClassifier(random_state=0)
    rf.fit(x_train, y_train)
    # test     
    pred_labels_rf  = rf.predict(x_test)
    score_rf = get_performance_df(y_test, pred_labels_rf,'Random Forest')   
    # train 
    pred_labels_rf_train  = rf.predict(x_train)
    score_rf_train = get_performance_df(y_train, pred_labels_rf_train,'Random Forest')         
    
    #Decision Tree
    dt = DecisionTreeClassifier()
    dt.fit(x_train, y_train)
    # test
    pred_labels_dt = dt.predict(x_test)
    score_dt = get_performance_df(y_test, pred_labels_dt,'Decision Tree')
    # train 
    pred_labels_dt_train  = dt.predict(x_train)
    score_dt_train = get_performance_df(y_train, pred_labels_dt_train,'Decision Tree')          

    #Gradient Boosting Classifier
    gb = GradientBoostingClassifier()
    gb.fit(x_train, y_train)
    # test 
    pred_labels_gb = gb.predict(x_test)
    score_gb = get_performance_df(y_test, pred_labels_gb,'Gradient Boosting Classifier')
    # train 
    pred_labels_gb_train  = gb.predict(x_train)
    score_gb_train = get_performance_df(y_train, pred_labels_gb_train,'Gradient Boosting Classifier')     
    
    # make dataframe with scores (on test data)
    scores = pd.concat(
        [score_knn, 
         score_lr, 
         score_bnb, 
         score_gnb, 
         score_rf, 
         score_dt, 
         score_gb]
    )
    scores = scores.sort_values(by = 'recall', ascending=False)
    
    # make dataframe with scores (on train data)
    scores_train = pd.concat(
        [score_knn_train, 
         score_lr_train, 
         score_bnb_train, 
         score_gnb_train, 
         score_rf_train, 
         score_dt_train, 
         score_gb_train]
    )
    scores_train = scores_train.sort_values(by = 'recall', ascending=False)    
        
    # make dataframe with predictions
    predictions = pd.DataFrame({
        'actuals':y_test,
        'pred_knn':pred_labels_knn,
        'pred_logistic_regression':pred_labels_lr,
        'pred_bernoulli_naive_bayes':pred_labels_bnb,
        'pred_gaussian_naive_bayes':pred_labels_gnb,
        'pred_random_forest':pred_labels_rf,
        'pred_decision_tree':pred_labels_dt,
        'pred_gradient_boosting_classifier':pred_labels_gb
    })

    return scores, predictions, scores_train

In [51]:
cor_columns = candidate_features.copy()
cor_columns.append('MDD')

cdc_survey_pmom['num_dep_screener_0'] = (cdc_survey_pmom[dep_screener_cols]==0).sum(axis=1)

# find columns where top feature is 0
#df_to_use = cdc_survey_pmom[cdc_survey_pmom['feeling_down_depressed_hopeless'] == 0]
df_to_use = cdc_survey_pmom[cdc_survey_pmom['num_dep_screener_0'] >= 9]

correlation_matrix = df_to_use[cor_columns].corr(method='pearson')
correlation_values_abs = abs(correlation_matrix['MDD']).sort_values(ascending=False)
correlation_values_abs = correlation_values_abs[correlation_values_abs.notna()]

top_additional_candidates_v2 = list(correlation_values_abs[1:30].index)
top_additional_candidates_v2 = [col for col in top_additional_candidates_v2 if col not in dep_screener_cols]
top_additional_candidates_v2

['times_with_12plus_alc',
 'seen_mental_health_professional',
 'count_days_seen_doctor_12mo',
 'count_lost_10plus_pounds',
 'arthritis',
 'horomones_not_bc',
 'is_usa_born',
 'times_with_8plus_alc',
 'time_since_last_healthcare',
 'duration_last_healthcare_visit',
 'work_schedule',
 'age_in_years',
 'regular_periods',
 'count_minutes_moderate_sedentary_activity',
 'emergency_food_received',
 'high_bp',
 'dr_recommend_exercise',
 'metal_objects',
 'drank_alc',
 'cholesterol_prescription',
 'smoked_100_cigs',
 'vigorous_recreation',
 'dr_recommend_lose_weight',
 'cancer',
 'chest_discomfort',
 'has_health_insurance',
 'have_health_insurance',
 'weight_lbs',
 'high_cholesterol']

In [52]:
print(model_features + [
    'readytoeat_meals',
    'regular_healthcare_place',
    'try_pregnancy_1yr',
    'currently_increase_exercise',
    'coronary_heart_disease',
    'stroke',
    'heart_attack',
    'see_dr_fertility'
])

['little_interest_in_doing_things', 'feeling_down_depressed_hopeless', 'trouble_falling_or_staying_asleep', 'feeling_tired_or_having_little_energy', 'poor_appetitie_or_overeating', 'feeling_bad_about_yourself', 'trouble_concentrating', 'moving_or_speaking_to_slowly_or_fast', 'thoughts_you_would_be_better_off_dead', 'difficult_doing_daytoday_tasks', 'times_with_12plus_alc', 'seen_mental_health_professional', 'count_days_seen_doctor_12mo', 'count_lost_10plus_pounds', 'arthritis', 'horomones_not_bc', 'is_usa_born', 'times_with_8plus_alc', 'time_since_last_healthcare', 'duration_last_healthcare_visit', 'work_schedule', 'age_in_years', 'regular_periods', 'count_minutes_moderate_sedentary_activity', 'emergency_food_received', 'high_bp', 'dr_recommend_exercise', 'metal_objects', 'drank_alc', 'cholesterol_prescription', 'smoked_100_cigs', 'vigorous_recreation', 'dr_recommend_lose_weight', 'cancer', 'chest_discomfort', 'has_health_insurance', 'have_health_insurance', 'weight_lbs', 'readytoeat_m

In [53]:

x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey_pmom,
    columns = model_features + [
    'readytoeat_meals',
    'regular_healthcare_place',
    'try_pregnancy_1yr',
    'currently_increase_exercise',
    'coronary_heart_disease',
    'stroke',
    'heart_attack',
    'see_dr_fertility'
]
)

processing_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'), RobustScaler(), GaussianNB())
      
processing_pipeline.fit(x_train, y_train)
y_pred  = processing_pipeline.predict(x_test)
# y_pred = [x.round() for x in y_pred]
df_gnb = pd.DataFrame(classification_report(
            y_test, 
            y_pred, 
            # target_names=target_names, 
            output_dict=True
))
df_gnb

,0,1,accuracy,macro avg,weighted avg
precision,0.954623,0.278215,0.78825,0.616419,0.885192
recall,0.802158,0.666667,0.78825,0.734412,0.788250
f1-score,0.871775,0.392593,0.78825,0.632184,0.822588
support,1390.000000,159.000000,0.78825,1549.000000,1549.000000


# Export GNB model to pickle

In [54]:
x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey_pmom,
    columns = model_features + [
                'readytoeat_meals',
                'regular_healthcare_place',
                'try_pregnancy_1yr',
                'currently_increase_exercise',
                'coronary_heart_disease',
                'stroke',
                'heart_attack',
                'see_dr_fertility'
            ]
)

baseline_models(
    x_train, 
    y_train, 
    x_test, 
    y_test, 
    algo_attempt_list=['gnb'],
    grid_search=True,
    cv=3,
    verbose=1,
    imputer=SimpleImputer(strategy='most_frequent'),#SimpleImputer(),
    scaler=RobustScaler(),
    bin_vars=True,
    score_on='recall',
    do_smote=False
    )

Fitting 3 folds for each of 100 candidates, totalling 300 fits


metric,algo,precision,recall,f1-score,support,accuracy,train_r2,test_r2,best_params,tp,fn,fp,tn
0,Gaussian Naive Bayes,0.616419,0.734412,0.632184,1549.0,0.78825,0.784238,0.78825,{'gaussiannb__var_smoothing': 1e-09},106,275,53,1115


# Brute Force

In [55]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import RepeatedStratifiedKFold




##### with all setting each trial takes a while ##########
##### commenting out options can make bring iteration time down drastically ###########
num_trials = 100
best_score = 0

primary_eval_metric = 'recall'
secondary_eval_metric = 'f1-score'
# primary_eval_metric = 'f1-score'
# secondary_eval_metric = 'recall'

best_cols = []
df_best_scores = pd.DataFrame()
df_all_scores = pd.DataFrame()
extra_selected_feats = []

try:
    
    for i in tqdm(range(1, num_trials+1)):
        random_feats_list = [col for col in candidate_features if col not in model_features+extra_selected_feats]
        random_feat = random_feats_list[random.randint(0, len(random_feats_list)-1)]
        
        if i == 1:
            feats_to_use = model_features + [
                # 'gallbladder_surgery'
                # , 'has_diabetes'
                # ,'readytoeat_meals'
                # , 'regular_healthcare_place'
                'try_pregnancy_1yr'
                , 'currently_increase_exercise'
                , 'coronary_heart_disease'
                , 'stroke'
                , 'heart_attack'
                , 'monthly_poverty_index'
                , 'see_dr_fertility'
                , 'high_bp_prescription'
            ]
            random_feat = None
        else:
            feats_to_use = model_features + [random_feat] +  [
                # 'gallbladder_surgery'
                # , 'has_diabetes'
                # ,'readytoeat_meals'
                # , 'regular_healthcare_place'
                'try_pregnancy_1yr'
                , 'currently_increase_exercise'
                , 'coronary_heart_disease'
                , 'stroke'
                , 'heart_attack'
                , 'monthly_poverty_index'
                , 'see_dr_fertility'
                , 'high_bp_prescription'
            ]

        x_train, x_test, y_train, y_test = get_model_data(
            original_df = cdc_survey_pmom,
            columns = feats_to_use
        )

        # SMOTE or not
        for b in [False]:#]:
            # different imputers
            for imp in [SimpleImputer(strategy='most_frequent')]:#[None, KNNImputer(), SimpleImputer(strategy='mean'), SimpleImputer(strategy='most_frequent')]:
                for scl in [RobustScaler()]:#[None, RobustScaler(), StandardScaler(), MinMaxScaler(), Normalizer()]:
                    for cv in [3]:#, 5, RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)]:
                        # print(b, imp, scl, cv)
                        df_scores = baseline_models(
                            x_train, 
                            y_train, 
                            x_test, 
                            y_test, 
                            algo_attempt_list=['gnb'],
                            grid_search=True,
                            cv=3,
                            verbose=0,
                            imputer=imp,
                            scaler=scl,
                            do_smote=b,
                            bin_vars=False,
                            score_on=primary_eval_metric
                            )

                        df_scores['SMOTE'] = b
                        df_scores['imputer'] = imp
                        df_scores['scaler'] = scl
                        df_scores['score_on'] = primary_eval_metric
                        df_scores['extra_random_feature'] = random_feat
                        df_scores['cv'] = cv
                        # sort to keep best
                        df_scores.sort_values(
                            by=[primary_eval_metric, secondary_eval_metric], 
                            inplace=True, 
                            ascending=False
                        )
                        df_scores = df_scores.reset_index()
                        df_scores = df_scores.iloc[:,1:]
                        df_all_scores = pd.concat([df_all_scores, df_scores], ignore_index=True)
                        if df_scores[primary_eval_metric][0] > best_score:
                            print(f'new best score is: {df_scores[primary_eval_metric][0]}')
                            print(f'got better with {random_feat}')
                            best_score = df_scores[primary_eval_metric][0]
                            extra_selected_feats.extend([random_feat])
                            best_f1 = df_scores[primary_eval_metric][0]
                            # best_cols = df_random_feats.columns
                            df_scores['features'] = str(df_scores.columns.tolist())
                            df_scores['trial_num'] = i
                            df_best_scores = pd.concat([df_best_scores, df_scores.iloc[0:1,:]], ignore_index=True)
except KeyboardInterrupt as e:
    print(e)

df_best_scores.sort_values(
    by=[primary_eval_metric, secondary_eval_metric], 
    inplace=True, 
    ascending=False
)
df_all_scores.sort_values(
    by=[primary_eval_metric, secondary_eval_metric], 
    inplace=True, 
    ascending=False
)

df_best_scores = df_best_scores.reset_index(drop=True)
print(best_f1)
print(best_cols)
print(extra_selected_feats)
df_best_scores

  1%|          | 1/100 [00:01<02:29,  1.51s/it]

new best score is: 0.734052757793765
got better with None


100%|██████████| 100/100 [02:34<00:00,  1.54s/it]

0.734052757793765
[]
[None]


metric,algo,precision,recall,f1-score,support,accuracy,train_r2,test_r2,best_params,tp,...,fp,tn,SMOTE,imputer,scaler,score_on,extra_random_feature,cv,features,trial_num
0,Gaussian Naive Bayes,0.616036,0.734053,0.6316,1549.0,0.787605,0.785368,0.787605,{'gaussiannb__var_smoothing': 2.31012970008315...,106,...,53,1114,False,SimpleImputer(strategy='most_frequent'),RobustScaler(),recall,None,3,"['algo', 'precision', 'recall', 'f1-score', 's...",1


In [56]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import RepeatedStratifiedKFold




##### with all setting each trial takes a while ##########
##### commenting out options can make bring iteration time down drastically ###########
num_trials = 100
best_score = 0

primary_eval_metric = 'recall'
secondary_eval_metric = 'f1-score'
# primary_eval_metric = 'f1-score'
# secondary_eval_metric = 'recall'

best_cols = []
df_best_scores = pd.DataFrame()
df_all_scores = pd.DataFrame()

try:
    
    for i in tqdm(range(1, num_trials+1)):
        random_feats_list = [col for col in candidate_features if col not in model_features]
        random_feat = random_feats_list[random.randint(0, len(random_feats_list)-1)]
        
        if i == 1:
            feats_to_use = model_features
            random_feat = None
        else:
            feats_to_use = model_features + [random_feat]

        x_train, x_test, y_train, y_test = get_model_data(
            original_df = cdc_survey_pmom,
            columns = feats_to_use
        )

        # SMOTE or not
        for b in [False]:#]:
            # different imputers
            for imp in [None]:#, KNNImputer(), SimpleImputer(strategy='mean'), SimpleImputer(strategy='most_frequent')]:
                for scl in [None]:#, RobustScaler(), StandardScaler(), MinMaxScaler(), Normalizer()]:
                    for cv in [3]:#, 5, RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)]:
                        # print(b, imp, scl, cv)
                        df_scores = baseline_models(
                            x_train, 
                            y_train, 
                            x_test, 
                            y_test, 
                            algo_attempt_list=['gnb'],
                            grid_search=True,
                            cv=3,
                            verbose=0,
                            imputer=imp,
                            scaler=scl,
                            do_smote=b,
                            bin_vars=False,
                            score_on=primary_eval_metric
                            )

                        df_scores['SMOTE'] = b
                        df_scores['imputer'] = imp
                        df_scores['scaler'] = scl
                        df_scores['score_on'] = primary_eval_metric
                        df_scores['extra_random_feature'] = random_feat
                        df_scores['cv'] = cv
                        # sort to keep best
                        df_scores.sort_values(
                            by=[primary_eval_metric, secondary_eval_metric], 
                            inplace=True, 
                            ascending=False
                        )
                        df_scores = df_scores.reset_index()
                        df_scores = df_scores.iloc[:,1:]
                        df_all_scores = pd.concat([df_all_scores, df_scores], ignore_index=True)
                        if df_scores[primary_eval_metric][0] > best_score:
                            best_f1 = df_scores[primary_eval_metric][0]
                            best_cols = df_random_feats.columns
                            df_scores['features'] = str(df_random_feats.columns.tolist())
                            df_scores['trial_num'] = i
                            df_best_scores = pd.concat([df_best_scores, df_scores.iloc[0:1,:]], ignore_index=True)
except KeyboardInterrupt as e:
    print(e)

df_best_scores.sort_values(
    by=[primary_eval_metric, secondary_eval_metric], 
    inplace=True, 
    ascending=False
)
df_all_scores.sort_values(
    by=[primary_eval_metric, secondary_eval_metric], 
    inplace=True, 
    ascending=False
)

df_best_scores = df_best_scores.reset_index(drop=True)
print(best_f1)
print(best_cols)
df_best_scores

  0%|          | 0/100 [00:00<?, ?it/s]


NameError: name 'df_random_feats' is not defined

# Best Single Model

In [ ]:
# x_train, x_test, y_train, y_test = get_model_data(
#     original_df = cdc_survey_pmom,
#     columns = dep_screener_cols + 
#               top_additional_candidates_v2[:28]
# )

# print(x_train.shape)
# print(x_test.shape)
# print(y_train.shape)
# print(y_test.shape)

# performance_test, predictions, performance_train = generate_models(
#     x_train, 
#     y_train, 
#     x_test, 
#     y_test
# )

# #baseline_model_summary_attempt1 = baseline_model_attempt1[baseline_model_attempt1['grouping'] == 'depressed_yes']
# performance_test_macro = performance_test[performance_test['grouping'] == 'macro_avg']
# performance_test_mdd1 = performance_test[performance_test['grouping'] == 'depressed_yes']
# performance_test_macro[['model','recall','f1-score','precision','accuracy']]